In [14]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2

# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library()  
spawn_points = world.get_map().get_spawn_points() 

In [2]:
import sys 
import os

In [3]:
dir_paths = ['CARLA_0.9.13\\WindowsNoEditor\\PythonAPI\\carla', 'scenario_runner']
sys.path += [os.path.join(os.getcwd(), dir_path) for dir_path in dir_paths]

In [6]:
for v in world.get_actors().filter('*vehicle*'):
    v.destroy()
for s in world.get_actors().filter('*sensor*'):
    s.destroy()

In [4]:
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [15]:
# Spawn ego vehicle
vehicle_bp = bp_lib.find('vehicle.tesla.model3') 
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# Move spectator behind vehicle to view
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

In [13]:
world = client.load_world('Town01_Opt')